In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import logging
import os

from data_prep import Password as P
from model import Generator, Discriminator
from training_helper import *
from config import *

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# custom weights initialization called on G and D
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0, 0.001)

In [3]:
g = Generator(GEN_HIDDEN_SIZE, GEN_NEURON_SIZE).to(device)
d = Discriminator(DISC_HIDDEN_SIZE, DISC_NEURON_SIZE).to(device)

In [4]:
#opt_g = torch.optim.Adam(g.parameters(), lr=0.0002, betas=(0.5, 0.999))
#opt_d = torch.optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_g = torch.optim.RMSprop(g.parameters(), lr=0.0002)
opt_d = torch.optim.RMSprop(d.parameters(), lr=0.0002)

In [5]:
p = P()
batch_gen = p.string_gen

INFO:root:Initializing passwords...
DEBUG:root:Loading from existing json file...
INFO:root:Done initializing passwords.


In [6]:
g.pre_train(p)

In [7]:
logger.setLevel(logging.DEBUG)

TRAIN_FROM_CKPT = False

if os.path.isfile(DEFAULT_CHECKPOINT_PATH) and TRAIN_FROM_CKPT:
    checkpoint = torch.load(DEFAULT_CHECKPOINT_PATH, map_location = device)
    g.load_state_dict(checkpoint['gen_state_dict'])
    d.load_state_dict(checkpoint['disc_state_dict'])
    g = g.to(device)
    d = d.to(device)
    opt_g.load_state_dict(checkpoint['gen_optimizer_state_dict'])
    opt_d.load_state_dict(checkpoint['disc_optimizer_state_dict'])
    start_len = checkpoint['seq_len']
    start_iter = checkpoint['iter']
    disc_loss = checkpoint['disc_loss']
    gen_loss = checkpoint['gen_loss']
else:
    start_len = 1
    start_iter = 1
    disc_loss = []
    gen_loss = []

for seq_len in range(start_len, MAX_LEN + 1):
    logging.info("---------- Adversarial Training with Seq Len %d, Batch Size %d ----------\n" 
                 % (seq_len, BATCH_SIZE))
    
    for i in range(start_iter, ITERS_PER_SEQ_LEN + 1):
                
        if i % SAVE_CHECKPOINTS_EVERY == 0:
            torch.save({
                'seq_len': seq_len,
                'gen_state_dict': g.state_dict(),
                'disc_state_dict': d.state_dict(),
                'gen_optimizer_state_dict': opt_g.state_dict(),
                'disc_optimizer_state_dict': opt_d.state_dict(),
                'iter': i,
                'gen_loss': gen_loss,
                'disc_loss': disc_loss
                }, DEFAULT_CHECKPOINT_PATH)
            logging.info("  *** Model Saved ***\n")
        
        logging.debug("----------------- %d / %d -----------------\n" % (i, ITERS_PER_SEQ_LEN))

        logging.debug("Training discriminator...\n")
        
        d.requiresGrad()
        d.zero_grad()
        g.zero_grad()
        for j in range(CRITIC_ITERS):
            with torch.backends.cudnn.flags(enabled=False):
                L = 0
                
                data = next(batch_gen)
                pred = g(data, seq_len)
                real, fake = get_train_dis(data, pred, seq_len)
                interpolate = get_interpolate(real, fake)

                # Genuine
                disc_real = d(real, seq_len)
                loss_real = -disc_real.mean()
                logging.debug("real loss: "+str(loss_real.item()))
                L += loss_real

                # Fake
                disc_fake = d(fake, seq_len)
                loss_fake = disc_fake.mean()
                logging.debug("fake loss: "+str(loss_fake.item()))
                L += loss_fake

                # Gradient penalty
                interpolate = torch.autograd.Variable(interpolate, requires_grad=True)
                disc_interpolate = d(interpolate, seq_len)
                grad = torch.ones_like(disc_interpolate).to(device)
                gradients = torch.autograd.grad(
                        outputs=disc_interpolate,
                        inputs=interpolate,
                        grad_outputs=grad,
                        create_graph=True,
                        retain_graph=True,
                        only_inputs=True,
                    )[0]
                loss_gp = ((gradients.norm(2, dim=2) - 1) ** 2).mean() * LAMBDA
                logging.debug("grad loss: "+str(loss_gp.item()))
                L += loss_gp

                L.backward(retain_graph=False)
                opt_d.step()
                
                logging.debug("Critic Iter " + str(j+1) + " Loss: " + str(L.item()) + "\n")


        logging.debug("Done training discriminator.\n")    

        logging.debug("Training generator...")

        d.requiresNoGrad()

        for j in range(GEN_ITERS):
            data = next(batch_gen)
            pred = g(data, seq_len)
            fake = get_train_gen(data, pred, seq_len)
            loss_gen = -d(fake, seq_len).mean()
            logging.debug("Gen Iter " + str(j+1) + " Loss: "+str(loss_gen.item()))
            loss_gen.backward(retain_graph=False)
            opt_g.step()

        logging.debug("Done training generator.\n")

        if i % SAVE_CHECKPOINTS_EVERY == 0:
            disc_loss.append(L)
            gen_loss.append(loss_gen)
    
    start_iter = 1
    

INFO:root:---------- Adversarial Training with Seq Len 1, Batch Size 16 ----------

DEBUG:root:----------------- 1 / 2000 -----------------

DEBUG:root:Training discriminator...

DEBUG:root:real loss: -0.016945943236351013
DEBUG:root:fake loss: 0.019754260778427124
DEBUG:root:grad loss: 8.695344924926758
DEBUG:root:Critic Iter 1 Loss: 8.698153495788574

DEBUG:root:real loss: -0.048288531601428986
DEBUG:root:fake loss: 0.039505768567323685
DEBUG:root:grad loss: 3.41825532913208
DEBUG:root:Critic Iter 2 Loss: 3.4094724655151367

DEBUG:root:real loss: -0.09266094863414764
DEBUG:root:fake loss: 0.167633056640625
DEBUG:root:grad loss: 0.11322176456451416
DEBUG:root:Critic Iter 3 Loss: 0.18819387257099152

DEBUG:root:real loss: -0.13249674439430237
DEBUG:root:fake loss: 0.1901552528142929
DEBUG:root:grad loss: 3.9395666122436523
DEBUG:root:Critic Iter 4 Loss: 3.997225046157837

DEBUG:root:real loss: -0.1104889065027237
DEBUG:root:fake loss: 0.07648900151252747
DEBUG:root:grad loss: 0.4820777

DEBUG:root:Gen Iter 2 Loss: 0.02873794548213482
DEBUG:root:Gen Iter 3 Loss: 0.031075458973646164
DEBUG:root:Gen Iter 4 Loss: 0.023904604837298393
DEBUG:root:Gen Iter 5 Loss: 0.010349457152187824
DEBUG:root:Gen Iter 6 Loss: 0.06347662210464478
DEBUG:root:Gen Iter 7 Loss: 0.04874962568283081
DEBUG:root:Gen Iter 8 Loss: 0.03494463860988617
DEBUG:root:Gen Iter 9 Loss: 0.058278098702430725
DEBUG:root:Gen Iter 10 Loss: 0.13602536916732788
DEBUG:root:Gen Iter 11 Loss: 0.030833229422569275
DEBUG:root:Gen Iter 12 Loss: 0.10472729802131653
DEBUG:root:Gen Iter 13 Loss: 0.002904844470322132
DEBUG:root:Gen Iter 14 Loss: 0.07589470595121384
DEBUG:root:Gen Iter 15 Loss: 0.06467974931001663
DEBUG:root:Gen Iter 16 Loss: 0.08510376513004303
DEBUG:root:Gen Iter 17 Loss: 0.01883668825030327
DEBUG:root:Gen Iter 18 Loss: 0.011870847083628178
DEBUG:root:Gen Iter 19 Loss: -0.033762913197278976
DEBUG:root:Gen Iter 20 Loss: 0.017754673957824707
DEBUG:root:Gen Iter 21 Loss: 0.04697544127702713
DEBUG:root:Gen Ite

DEBUG:root:----------------- 8 / 2000 -----------------

DEBUG:root:Training discriminator...

DEBUG:root:real loss: -0.05417267233133316
DEBUG:root:fake loss: -0.13072790205478668
DEBUG:root:grad loss: 0.0036052940413355827
DEBUG:root:Critic Iter 1 Loss: -0.18129529058933258

DEBUG:root:real loss: -0.05565563216805458
DEBUG:root:fake loss: -0.215803325176239
DEBUG:root:grad loss: 0.0033825081773102283
DEBUG:root:Critic Iter 2 Loss: -0.26807644963264465

DEBUG:root:real loss: -0.04034539312124252
DEBUG:root:fake loss: -0.15596789121627808
DEBUG:root:grad loss: 0.0024510722141712904
DEBUG:root:Critic Iter 3 Loss: -0.1938622146844864

DEBUG:root:real loss: 0.01984488219022751
DEBUG:root:fake loss: -0.2049514204263687
DEBUG:root:grad loss: 0.002688955282792449
DEBUG:root:Critic Iter 4 Loss: -0.1824175864458084

DEBUG:root:real loss: -0.04920428246259689
DEBUG:root:fake loss: -0.12432318925857544
DEBUG:root:grad loss: 0.0029591212514787912
DEBUG:root:Critic Iter 5 Loss: -0.1705683618783950

DEBUG:root:Gen Iter 4 Loss: 0.23861069977283478
DEBUG:root:Gen Iter 5 Loss: 0.1987733542919159
DEBUG:root:Gen Iter 6 Loss: 0.22877052426338196
DEBUG:root:Gen Iter 7 Loss: 0.18308305740356445
DEBUG:root:Gen Iter 8 Loss: 0.2289058417081833
DEBUG:root:Gen Iter 9 Loss: 0.21804669499397278
DEBUG:root:Gen Iter 10 Loss: 0.12760013341903687
DEBUG:root:Gen Iter 11 Loss: 0.23501071333885193
DEBUG:root:Gen Iter 12 Loss: 0.1252806931734085
DEBUG:root:Gen Iter 13 Loss: 0.16221067309379578
DEBUG:root:Gen Iter 14 Loss: 0.242695152759552
DEBUG:root:Gen Iter 15 Loss: 0.21513372659683228
DEBUG:root:Gen Iter 16 Loss: 0.17592494189739227
DEBUG:root:Gen Iter 17 Loss: 0.2083154022693634
DEBUG:root:Gen Iter 18 Loss: 0.2241573929786682
DEBUG:root:Gen Iter 19 Loss: 0.17898470163345337
DEBUG:root:Gen Iter 20 Loss: 0.23409713804721832
DEBUG:root:Gen Iter 21 Loss: 0.22425368428230286
DEBUG:root:Gen Iter 22 Loss: 0.25431302189826965
DEBUG:root:Gen Iter 23 Loss: 0.2161521166563034
DEBUG:root:Gen Iter 24 Loss: 0.262

DEBUG:root:fake loss: -0.164174884557724
DEBUG:root:grad loss: 0.003489724127575755
DEBUG:root:Critic Iter 1 Loss: -0.24979306757450104

DEBUG:root:real loss: -0.10776121914386749
DEBUG:root:fake loss: -0.16991978883743286
DEBUG:root:grad loss: 0.004250331781804562
DEBUG:root:Critic Iter 2 Loss: -0.2734306752681732

DEBUG:root:real loss: -0.08877122402191162
DEBUG:root:fake loss: -0.08273699134588242
DEBUG:root:grad loss: 0.004767295904457569
DEBUG:root:Critic Iter 3 Loss: -0.16674092411994934

DEBUG:root:real loss: -0.0108218127861619
DEBUG:root:fake loss: -0.2969213128089905
DEBUG:root:grad loss: 0.0035331090912222862
DEBUG:root:Critic Iter 4 Loss: -0.30421003699302673

DEBUG:root:real loss: -0.021791070699691772
DEBUG:root:fake loss: -0.2624548673629761
DEBUG:root:grad loss: 0.003824823535978794
DEBUG:root:Critic Iter 5 Loss: -0.28042110800743103

DEBUG:root:Done training discriminator.

DEBUG:root:Training generator...
DEBUG:root:Gen Iter 1 Loss: 0.30157649517059326
DEBUG:root:Gen 

...

DEBUG:root:Gen Iter 7 Loss: 0.4137685298919678
DEBUG:root:Gen Iter 8 Loss: 0.25021904706954956
DEBUG:root:Gen Iter 9 Loss: 0.312386691570282
DEBUG:root:Gen Iter 10 Loss: 0.32845041155815125
DEBUG:root:Gen Iter 11 Loss: 0.2541313171386719
DEBUG:root:Gen Iter 12 Loss: 0.32399338483810425
DEBUG:root:Gen Iter 13 Loss: 0.3152700364589691
DEBUG:root:Gen Iter 14 Loss: 0.27569878101348877
DEBUG:root:Gen Iter 15 Loss: 0.30419909954071045
DEBUG:root:Gen Iter 16 Loss: 0.28067293763160706
DEBUG:root:Gen Iter 17 Loss: 0.3434655964374542
DEBUG:root:Gen Iter 18 Loss: 0.21247407793998718
DEBUG:root:Gen Iter 19 Loss: 0.30167585611343384
DEBUG:root:Gen Iter 20 Loss: 0.3609367609024048
DEBUG:root:Gen Iter 21 Loss: 0.24867863953113556
DEBUG:root:Gen Iter 22 Loss: 0.27100348472595215
DEBUG:root:Gen Iter 23 Loss: 0.27158164978027344
DEBUG:root:Gen Iter 24 Loss: 0.2336738407611847
DEBUG:root:Gen Iter 25 Loss: 0.40186870098114014
DEBUG:root:Done training generator.

DEBUG:root:----------------- 220 / 2000 ---

DEBUG:root:fake loss: -0.38647693395614624
DEBUG:root:grad loss: 0.016620300710201263
DEBUG:root:Critic Iter 2 Loss: -0.45626184344291687

DEBUG:root:real loss: -0.08647221326828003
DEBUG:root:fake loss: -0.28973251581192017
DEBUG:root:grad loss: 0.03681425750255585
DEBUG:root:Critic Iter 3 Loss: -0.33939045667648315

DEBUG:root:real loss: -0.13968005776405334
DEBUG:root:fake loss: -0.21704766154289246
DEBUG:root:grad loss: 0.025257743895053864
DEBUG:root:Critic Iter 4 Loss: -0.33146998286247253

DEBUG:root:real loss: -0.12281690537929535
DEBUG:root:fake loss: -0.27938157320022583
DEBUG:root:grad loss: 0.014543497003614902
DEBUG:root:Critic Iter 5 Loss: -0.38765498995780945

DEBUG:root:Done training discriminator.

DEBUG:root:Training generator...
DEBUG:root:Gen Iter 1 Loss: 0.39540529251098633
DEBUG:root:Gen Iter 2 Loss: 0.284880131483078
DEBUG:root:Gen Iter 3 Loss: 0.261447548866272
DEBUG:root:Gen Iter 4 Loss: 0.35612350702285767
DEBUG:root:Gen Iter 5 Loss: 0.3782830834388733
DEBUG:r

DEBUG:root:Gen Iter 12 Loss: 0.3712763488292694
DEBUG:root:Gen Iter 13 Loss: 0.3414790630340576
DEBUG:root:Gen Iter 14 Loss: 0.41862866282463074
DEBUG:root:Gen Iter 15 Loss: 0.39698970317840576
DEBUG:root:Gen Iter 16 Loss: 0.3514033854007721
DEBUG:root:Gen Iter 17 Loss: 0.3676903545856476
DEBUG:root:Gen Iter 18 Loss: 0.26689037680625916
DEBUG:root:Gen Iter 19 Loss: 0.36831408739089966
DEBUG:root:Gen Iter 20 Loss: 0.3611304759979248
DEBUG:root:Gen Iter 21 Loss: 0.2341683804988861
DEBUG:root:Gen Iter 22 Loss: 0.3622075319290161
DEBUG:root:Gen Iter 23 Loss: 0.34511277079582214
DEBUG:root:Gen Iter 24 Loss: 0.23166146874427795
DEBUG:root:Gen Iter 25 Loss: 0.4167802929878235
DEBUG:root:Done training generator.

DEBUG:root:----------------- 227 / 2000 -----------------

DEBUG:root:Training discriminator...

DEBUG:root:real loss: -0.06522862613201141
DEBUG:root:fake loss: -0.32602959871292114
DEBUG:root:grad loss: 0.03289531171321869
DEBUG:root:Critic Iter 1 Loss: -0.35836291313171387

DEBUG:r

DEBUG:root:Critic Iter 3 Loss: -0.3162861764431

DEBUG:root:real loss: -0.08314177393913269
DEBUG:root:fake loss: -0.3136463463306427
DEBUG:root:grad loss: 0.06925266236066818
DEBUG:root:Critic Iter 4 Loss: -0.3275354504585266

DEBUG:root:real loss: -0.08774339407682419
DEBUG:root:fake loss: -0.3602713346481323
DEBUG:root:grad loss: 0.04407714307308197
DEBUG:root:Critic Iter 5 Loss: -0.40393757820129395

DEBUG:root:Done training discriminator.

DEBUG:root:Training generator...
DEBUG:root:Gen Iter 1 Loss: 0.29825013875961304
DEBUG:root:Gen Iter 2 Loss: 0.2345542162656784
DEBUG:root:Gen Iter 3 Loss: 0.4079638123512268
DEBUG:root:Gen Iter 4 Loss: 0.34548452496528625
DEBUG:root:Gen Iter 5 Loss: 0.531603217124939
DEBUG:root:Gen Iter 6 Loss: 0.2747460901737213
DEBUG:root:Gen Iter 7 Loss: 0.442884624004364
DEBUG:root:Gen Iter 8 Loss: 0.404146283864975


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

logger.setLevel(logging.INFO)

plt.figure()
plt.plot(loss_trend)
#plt.savefig('image/rnn-rnn-loss.png',dpi=400)
plt.show()

In [ ]:
print_every = 100
loss_trend = []
logging.info("---------- Pre-training generator ----------")
for i in range(PRE_GEN_ITERS):
    pas = next(batch_gen)
    input_tensor = P.passwordToInputTensor(pas).to(device)
    target_tensor = P.passwordToTargetTensor(pas).to(device)
    output, loss = g.pre_train(input_tensor, target_tensor)
    
    if i % print_every == 0:
        loss_trend.append(loss)
        logging.debug("Iter: "+ str(i)+" Loss: "+str(loss))

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)

In [18]:
gr = nn.GRU(5, 6, 1, batch_first = True, dropout = 0).to(device)
inp = torch.ones(2,3,5).to(device)
print(inp.dim())
le = [2,3]
inp = nn.utils.rnn.pack_padded_sequence(inp, le, batch_first=True, enforce_sorted=False)
ii, batch_sizes, sorted_indices, unsorted_indices = inp
output = gr(inp)
batch_sizes, ii.dim()

3


(tensor([2, 2, 1]), 2)

In [6]:
checkpoint = torch.load(DEFAULT_CHECKPOINT_PATH, map_location = device)
g.load_state_dict(checkpoint['gen_state_dict'])
d.load_state_dict(checkpoint['disc_state_dict'])

<All keys matched successfully>

In [53]:
g.pre_train(p)

In [7]:
print(g.generate_N(p))

['6555555555555555555', '18utllfffffffffffff', '0926222222222222222', 'G111818111111111111', 'askyingzin', 'beydhia1', '8766111111111111111', '321185', 'ERAS000000000000000', 'Jorandamy-111111111', 'seyhanuomshooF111', 'ttakn62', '2873000000000000000', 'ruminndcybroFu', 'SALYwLGEMAw1', 'danadchorF1', 'shumissh22222222222', '0073000000000000000', 'lelanaaaaaaaaaaaaaa', '4eeyhorF1']


In [55]:
d.pre_train(p, g)

In [18]:
print(test(d, "pas"))

tensor([[731.1170]], device='cuda:0', grad_fn=<AddmmBackward>)


In [17]:
def test(d, password):
    tensor = P.passwordToPretrainTensor(password).float()
    tensor = torch.cat((torch.zeros(1, 10, CHARMAP_LEN).to(device), tensor), dim = 1)
    tensor = d.embedding(tensor)
    hidden = torch.zeros(d.layers, 1, d.hidden_size).to(device)
    out, _ = d.gru(tensor, hidden)
    loss = d.h2o(out[:,-1,:])
    return loss

In [18]:
data = next(batch_gen)
pred = g(data, 8)
real, fake = get_train_dis(data, pred, 8)
#d(real, 8)
#d(fake, 8)
fake[16]
#real[0]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000, 

In [10]:
disc_loss

[tensor(-0.3150, device='cuda:0', grad_fn=<AddBackward0>)]